Для работы с фото мы будем использовать пакет `cv2`. Для его установки нужно прописать в консоли `pip3 install opencv-python`.  Для него написана довольно хорошая и понятная [документация.](http://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_gui/py_image_display/py_image_display.html) 

In [ ]:
import pandas as pd    # Пакет для работы с таблицами
import numpy as np     # Пакет для работы с векторами и матрицами
import pickle          # Пакет для сохранения и подгрузки данных 
import urllib          # Пакет для чтения ссылок
   
# Пакет для красивых циклов. При желании его можно отключить и удалить из всех циклов 
# команду tqdm_notebook.
from tqdm import tqdm_notebook

import cv2 # Пакет для работы с фоточками 

from matplotlib import pyplot as plt  # Графики 
%matplotlib inline

Подгружаем вектор из фотографий. По каждому фото у нас есть информация о том как давно она залита вк, какая у неё высота, ширина, ссылка, по каким координатам обрезана фотка. 

In [ ]:
with open('../vk_data/vk_data_photos_v1', 'rb') as f:
    photo = pickle.load(f)

photo[10]

Отлично, фото в наших руках. На самом деле каждая картинка это набор пикселей. Если мы попросим питон показать нам картинку, он покажет матрицу из чисел.  Каждому пикселю в этой матрице соответствует число. Это число сообщает нам о том насколько этот пиксель яркий. Яркость измеряется по шкале от $0$ до $255$.

In [ ]:
img[0][:5]  # цвета верхник пикселей очень даже белые

Цветные картинки представляются в виде [тензора,](https://www.wikiwand.com/ru/%D0%A2%D0%B5%D0%BD%D0%B7%D0%BE%D1%80) то есть матрицы из матриц. Любой цвет можно получить, смешав в какой-то пропорции красный, зелёный и синий цвета. В связи с этим каждый пиксель обычно характеризуют тремя цифрами: (насколько пиксель красный, насколько пиксель зелёный, насколько пиксель синий). Такой формат хранения картинки называется [RGB-форматом.](https://www.wikiwand.com/ru/RGB)


In [ ]:
# Видим, что у нас картинка размера 87 на 130 пикселей и описывается 3 матрицами (одна на каждый цвет)
img.shape 

Все действия по редактированию картинки сводятся к математике. Чтобы осветлить картинку, нужно прибавить к каждому пикселю какое-то число. Для этого используют функцию `cv2.add`. В случае прибавления очень большого числа, она накопит яркость 255 и не пробьёт этот порог.

In [ ]:
img_1 = cv2.add(img,100)
plt.imshow(img_1)
plt.axis("off")

Умножение каждого пикселя на какое-то число увеличит контраст.

In [ ]:
img_1 = cv2.multiply(img,2) # повышаем контраст
plt.imshow(img_1)
plt.axis("off")

Можно посмотреть на то как распределены на картинке пиксели различной яркости.

In [ ]:
color = ('b','g','r')
for i,col in enumerate(color):
    histr = cv2.calcHist([img],[i],None,[256],[0,256])
    plt.plot(histr,color = col)
    plt.xlim([0,256])
plt.show()

In [ ]:
P = df_phe['photo']

# Посмотрим на несколько рандомных аватарок 
plt.title('sample image')
for i in range(6):
    plt.subplot(2,3,i+1)
    plt.imshow(P[i], cmap = 'gray')


Перейдём к фишкам, которые используют под капотом все фоторедакторы, а также свёрточные нейронные сетки. 

## 2. Свёртка

Свёртка это операция, которая превращает набор одних пикселей в другие. Обычно она осущствляется с помощью ядра свёртки, матрицы произвольного размера (обычно 3х3). Центральный элемент такой матрицы называется якорем свёртки. Он применяется к центральному пикселю. 

Работает свёртка очень просто. При вычислении нового значения выбранного пикселя изображения, ядро свёртки прикладывается своим центром к этому пикселю. Далее, вычисляется сумма произведений значений пикселей изображения на значения, накрывшего данный пиксель элемента ядра. Полученная сумма и является новым значением выбранного пикселя.

Используя матрицы с разными коэффициентами можно получать раные эффекты. 

Попробуем ухудшить качество изображения. В этом нам поможет следующее ядро размера 3 на 3.

$$ K = \frac{1}{9} \cdot \begin{pmatrix}
1 & 1 & 1  \\
1 & 1 & 1  \\         
1 & 1 & 1 
\end{pmatrix} $$

Оно берёт пиксель в каждом квадрате размера 3 на 3 и заменяет его на арифмитическое среднее всех пикселей. Таким образом размерность картинки и её качество падают.

In [ ]:
plt.figure(figsize=(10,10))

kernel = np.ones((3,3),np.float32)/9  # Создали в нумпай матрицу из 1/9 размера 3 на 3
dst = cv2.filter2D(img,-1,kernel)     # применили матрицу к нашей картинке 

plt.subplot(121),plt.imshow(img,),plt.title('Original')
plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(dst),plt.title('Averaging')
plt.xticks([]), plt.yticks([])

Попробуем применить другие ядра. Например, ядро для сглаживания.

$$ \begin{pmatrix}
0.1 & 0.1 & 0.1  \\
0.1 & 0.1 & 0.1 \\         
0.1 & 0.1 & 0.1 
\end{pmatrix} $$

In [ ]:
plt.figure(figsize=(10,10))

kernel = 0.05*np.ones((3,3),np.float32)
dst = cv2.filter2D(img,-1,kernel)

plt.subplot(121),plt.imshow(img,),plt.title('Original')
plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(dst),plt.title('Averaging')
plt.xticks([]), plt.yticks([])

Ядро для увеличения чёткости. Обратите внимание на большое значение якоря.

$$ \begin{pmatrix}
0.1 & 0.1 & 0.1  \\
0.1 & 2 & 0.1 \\         
0.1 & 0.1 & 0.1 
\end{pmatrix} $$

In [ ]:
plt.figure(figsize=(10,10))

kernel = -0.1*np.ones((3,3),np.float32)
kernel[1,1]=2
dst = cv2.filter2D(img,-1,kernel)

plt.subplot(121),plt.imshow(img,),plt.title('Original')
plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(dst),plt.title('Averaging')
plt.xticks([]), plt.yticks([])

Ядра бывают очень разными. Например, вы можете попробовать использовать следующие: 

* Ядро для увеличения яркости. 

$$ \begin{pmatrix}
-0.1 & 0.2 & -0.1  \\
0.2 & 3 & 0.2 \\         
-0.1 & 0.2 & -0.1 
\end{pmatrix} $$

* Ядро для затемнения. 

$$ \begin{pmatrix}
-0.1 & 0.1 & -0.1  \\
 0.1 & 0.5 & 0.1 \\         
-0.1 & 0.1 & -0.1 
\end{pmatrix} $$

* Ядро, которое ничего не делает 

$$ \begin{pmatrix}
0 & 0 & 0  \\
0 & 1 & 0 \\         
0 & 0 & 0 
\end{pmatrix} $$

* Ядро, которое сдвигает картинку

$$ \begin{pmatrix}
1 & 0 & 100  \\
0 & 1 & 50     
\end{pmatrix} $$

* Ядро для пофорота картинки на угол $\phi$ 

$$ \begin{pmatrix}
\cos\phi & -sin\phi  \\
\sin\phi & \cos\phi
\end{pmatrix} $$

* Эрозия и наращивание. Выбираем пиксель с максимальной или минимальной интенсивностью из окрестности.  Наращиваение приводит к увеличению ярких объектов, а эрозия к увеличению тёмных. Наращивание может быть использовано для увеличения бликов ярких изображений. Обычно эрозия имеет округлую форму и выглядит, например, так:

$$ \begin{pmatrix}
0 & 0 & 1 & 0 & 0  \\
0 & 1 & 1 & 1 & 0  \\         
1 & 1 & 1 & 1 & 1  \\
0 & 1 & 1 & 1 & 0  \\        
0 & 0 & 1 & 0 & 0
\end{pmatrix} $$


Абсолютно любая матрица задаёт какое-то преобразование линейного пространства. В данном случае нашей фотографии. При хорошем знании линала, вы можете придумать самые безумные ядра. Попробуйте! 

## 3. Углубляемся в свёртку

Попробуйте угадать, что делают следующие два фильтра. 

$$ \begin{pmatrix}
-1 & -1 & -1  \\
0 & 0 & 0 \\         
1 & 1 & 1 
\end{pmatrix}  $$

$$ \begin{pmatrix}
-1 & 0 & 1  \\
-1 & 0 & 1 \\         
-1 & 0 & 1 
\end{pmatrix} $$

Внимание, ответ. Первый фильтр пытается понять насколько резко изменяется яркость картинки по вертикали и находит вертикальные границы. Второй фильтр пытается понять насколько резко изменяется картинка по горизонтали и находит горизонтальные границы. Если просуммировать применение этих фильтров, можно получить чёткое очертание границ картинки. Чем больше в матрицах цифры, тем более резкую разницу находят ядра. 

In [ ]:
plt.figure(figsize=(12,12))

kernel1 = np.array([[-1,-1,-1],[0,0,0],[1,1,1]],np.float32)
kernel2 = kernel1.T

dst1 = cv2.filter2D(img,-1,kernel1)
dst2 = cv2.filter2D(img,-1,kernel2)
gr1 = cv2.add(dst1,dst2)

plt.subplot(131),plt.imshow(dst1),plt.title('vertical')
plt.xticks([]), plt.yticks([])
plt.subplot(132),plt.imshow(gr1),plt.title('All')
plt.xticks([]), plt.yticks([])
plt.subplot(133),plt.imshow(dst2),plt.title('horisontal')
plt.xticks([]), plt.yticks([])

Вроде бы граница получилась более чёткой. Внутри пакета есть своя функция для выделения границы. Он работает более агрессивно нежели наше ядро.

In [ ]:
plt.figure(figsize=(10,10))

kernel1 = np.array([[-1,-1,-1],[0,0,0],[1,1,1]],np.float32)
kernel2 = kernel1.T

dst1 = cv2.filter2D(img,-1,kernel1)
dst2 = cv2.filter2D(img,-1,kernel2)
my_gr = cv2.add(dst1,dst2)

# Пакетная функция:
its_gr = cv2.Canny(img,100,200)

plt.subplot(121),plt.imshow(my_gr),plt.title('All')
plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(its_gr,cmap='gray'),plt.title('horisontal')
plt.xticks([]), plt.yticks([])

Вообще, такое ядро задаёт градиент картики. Можно побаловаться с более крутыми градиентами, в том числе диагональными.

$$ \begin{pmatrix}
0 & 1 & 2  \\
-1 & 0 & 1 \\         
-2 & -1 & 0 
\end{pmatrix} $$

Как уже догадался читатель, можно придумать фильры не только для поиска границ, но и для поиска других различных закономерностей, которые есть на картинке. 

Представим себе на секунду славный дивный мир, в котором бывают картинки только двух типов: с прямыми слэшами и с обратными (/ \). При этом, эти слэши могут быть нарисованы на картинке где угодно. Пусть у нас есть две картинки. На одной из них слэш нарисован внизу справа, на второй сверху слева. Пройдёмся по нашим картинкам специальным ядром, которое ищет обратные слэши. После свёртки, мы получим на выходе две уменьшившиеся в размерах картинки, в каждой из которых будет фигурировать цифра два, как раз отвечающая за найденный обратный слэш.

<img align="center" src="photo_1.png" height="500" width="500"> 

Если точно такое же ядро натравить на картинку без обратного слэша, оно не выдаст нам на выход никакой двойки. 

<img align="center" src="photo_2.png" height="600" width="600"> 

Получаем простейший классификатор картинок с слэшами. 

1. Проходимся по картинке ядром. 
2. Находим в итоговой матрице максимальный элемент.
3. Если это двойка, на картинке изображён слэш. Если это единица, на картинке обратный слэш.

Обратите внимание, что работа этого классификатора не зависит от того, где именно на картинке находится слэш. Именно так свёрточные нейронные сети и работают с картинками. Конечно же, в реальности закономерности на картинках на порядок сложнее. При этом, мы даже не знаем какими именно могут быть эти закономерности. Для того, чтобы их искать, в нейронную сетку добавляются ядра с неспецифицированными параметрами.

$$ \begin{pmatrix}
w_1 & w_2 & w_3  \\
w_4 & w_5 & w_6 \\         
w_7 & w_8 & w_9 
\end{pmatrix} $$

Слои, на которых находятся эти ядра называются свёрточными. Параметры подбираются в ходе обучения нейронной сети по реальным данным и отражают в себе какие-то закономерности, найденные во время обучения на картинках. Представим, что наша нейронная сеть должна уметь распознавать лица. Добавим в неё несколько свёрточных слоёв. 

Первый слой будет находить простейшие элементы, такие как слэши, прямые чёрточки и извилистые чёрточки. Второй слой будет конструировать из элементов, найденных на первом слое, ещё более сложные штуки. В данном случае окружности и крестики. Третий слой будет конструировать из объектов, найденных на втором слое ещё более сложные объекты. Таким образом, мы, слой за слоем, будем собирать всё более и более сложные объекты до тех пор, пока не дойдём до лица. 

<img align="center" src="photo_3.png" height="600" width="600"> 

Обычно закономерности, которые находит свёрточная сеть сложно интерпретируемы. Тем не менее, мы можем забрать те закономерности, которые нашла нейросетка и использовать их в качестве регрессоров в какой-то более интерпретируемой модели.

Например, пусть перед нами стоит задача оценить стоимость поездки на такси в зависимости от времени суток, расстояния и кучи других параметров. Представим, что существует закон, который обязывает пассажиров такси фотографировать своего водителя перед каждой поездкой и после каждой поездки. С помощью нейронок мы могли бы извлечь из этих фотографий для нашей интерпретируемой линейной регрессии дополнительные квазиинтерпретируемые фичи. Полюбак нейросеть вытащит фичи, которые будет отвечать за то, какая у водителя национальность, возраст, черты лица и т.п. Вполне может оказаться, что какие-нибудь редкие армяне-альбиносы гоняют быстрее всех. Было бы забавно посмотреть на город, где был бы принят такой закон. Молодая дама вызывает такси, приезжает водитель, она фоткает его. Нейросеть в её телефоне определяет особенности этого водителя и выясняет насколько дорого будет стоить поездка, и дама отказывается ехать под предлогом того, что нейросеть показывает очень большую прогнозную стоимость. Абсурд? Ничего подобного!

Конечно же нейросети работают немножечко сложнее. Конечно же в них намного больше различных слоёв и для каждой, решаемой с помощью них задачи, существует огромная куча тонкостей. Например, отдельный вопрос состоит в том как должен вести себя алгоритм свёртки на краях изображения. Выше мы применили ядро к картинке со слэшем. Это уменьшило её с размера $4 \times 4$ до размера $3 \times 3$. Обычно для решения этой проблемы, на краях добавляются дополнительные ряды из пикселей, заполненных нулями так, чтобы размер картинки при свёртке не менялся. 

Размер картинок обычно умешьшают на отдельных слоях с помощью штуки, которую называют пулинг. Из всех квадратов размера $2 \times 2$ либо берётся максимальный элемент, либо среднее. Это позволяет уменьшить изображение. Будем забивать себе голову всякими тонкостями постепенно и перейдём чуть ближе к делу. Напишем несколько нейросеток. 

## 1. Про нейросети. 

### 1.1 Немного истории 

Сегодня мы все с вами живём в эпоху революции в области машинного обучения. Она началась примерно 10 лет назад и это ни для кого не секрет. Эта революция напрямую связана с третьей (первой успешной) реинкарнацией нейронных сетей.

Несмотря на весь тот хайп, которым сегодня окутаны нейронные сети, это один из старейших алгоритмов машинного обучения. Человечество постоянно пыталось использовать нейросети, но это начало у него получаться совсем недавно. 

Первая модель нейрона была предложена в 1943 году Уорреном Маккалоком и Уолтером Питсом. Уже в 1958 году Фрэнк Розенблатт предложил первую, самую простую нейросеть, которая могла разделять объекты в двухмерном пространстве. 

В 1960-е годы интерес к сеткам был довольно высоким, но по мере появления других, более хороших алгоритмов, угас. В 1990-е интерс к нейронкам снова возрос. Три незвысимые группы учёных одновременно допёрли до алгоритма обрататного распространения ошибки (backpropagation), который позволил обучать нейронки на порядок быстрее (сложность обучения упала с $O(n^2)$ до $O(n)$. Чуть погодя интерес к нейронкам снова угас. 

Сегодня мы снова живём в эпоху нового ренессанса нейросетей. Неожиданно выяснилось, что в отличие от большинства алгоритмов нейросети очень критичны к объёму данных. На маленьких объёмах данных из-за своей сложной структуры они очень плохо работают. Вместо того, чтобы выявить закономерность и обобщить её, они просто-напросто запоминают выборки. К сегодняшнему дню человечество подкопило данных и продолжает накапливать экспоненциальными темпами. 

На огромных объёмах данных нейросети работают очень даже хорошо. Более того, оказалось, что нейросети способны решать такие задачи, которые раньше считались нерешаемыми. Например, они умеют подписывать изображения, переносить стили художников на другие изображения и многое другое. В блажайшие годы сфера применения нейросетей будет только расширяться. Область развивается настолько динамично, что научные статьи за год успевают устареть. 

Несмотря на всю эту мощь, на современном этапе развития существует ряд серьёзных проблем, с которыми человечеству придётся разбираться.

Например, в январе 2017 года был проведён следующий эксперимент. Учёные мужи взяли огромную выборку из картинок. Для каждой картинки было подписано что именно на ней изображено. Все картинки и правильные ответы перемешали. Панда с одной картинки стала поездом, а панда с другой картики стала львом. Любые закономерности, которые можно было бы использовать для распознавания картинок, исчезли. После на эти данные были натравлены нейросетки. Оказалось, что все сетки достигают на огромной обучающей выборке нулевой ошибки, то есть просто-напросто запоминают выборку и никак не сообщают нам о том, что никакой закономерности в данных не было. При всём этом, нейросеть уверенно выдавала прогнозы для новых картинок и, конечно же, её точность была сравнима со случайным угадыванием. 

Другой открытой проблемой является то, что нейросетки принимают какие-то решения с какой-то точностью, но все эти решения неинтерпретируемы. Люди, в свою очередь, хотели бы понимать почему компьютер принял то или иное решение. Когда мы распознаём картинки, это не так критично, но когда речь идёт о медицинском диагнозе или о поведении на дороге самоуправляемой машины, нам хотелось бы понимать почему сетка приняла именно такое решение и насколько она в нём уверена. Если уверенность машины в своём поведении падает, надо переходить на ручное управление. На современном этапе своего развития, сетки не сообщают о том насколько они уверены в своём решении.  

Более того, работу нейросетей можно дестабилизировать. Например, если нарисовать на дорожном знаке маркером небольшую загогулину, нейросеть, вшитая в нашу машину, может неожиданно решить, что это вовсе не дорожный знак, а олень. 

Сегодня в направлении решения этих проблем идёт активная работа. Перспективной кажется ещё более древняя нежели нейросети идея, которая была предложена Томасом Байесом в далеком 18 веке. Байесовская статистика поднобно нейронным сетям переживает сегодня серьёзный ренессанс. Байесовские методы как и нейросети требуют огромных вычислительных ресурсов. Весь этот ренессанс и новые крутые технологии стали возможны благодаря новым компьютерам, появившимся у людей только в начале нового тысячелетия. Байесовские методы позволяют воспитать целый континнум нейронных сетей. Сделав выборку сетей из этого континума, можно оценить то насколько сетки уверены в своём решении. Такую технологию люди придум

но о ней мы поговорим в другой раз. Наверное, отдельно стоит отметить, что и нейронки и байесовские методы требуют огромных вычислительных ресурсов, которые появились у человечества только в начале нового тысячилетия.  

Ну и, наверное, последнее, что стоит сказать в этом небольшом экскурсе, это то, что нейросеть это далеко не искуственный интеллект. В основе нейронок лежит самая простая модель нейрона. Человеческий мозг устроен на порядок сложнее. Чуть ниже станет понятно, что нейросеть не особо сильно отличается от обычной линейной регрессии и все слухи о том, что возникновение скайнет возможно уже завтра, всего лишь слухи. До настоящей скайнет человечеству ещё далеко. Но, в прочем, подробнее об этой стороне вопроса можно узнать из очень крутой книги Педро Домингоса, [Верховный Алгоритм.](https://www.ozon.ru/context/detail/id/136780904/)



### 1.2 Как работают нейронки 















В принципе это вся информация, которую я хотел бы здесь описать. Не очень хочется грузить читателя моего небольшого научпопа излишними деталями. Детали оставим для курсов по машинному обучению. Тем не менее, если у вас возникло желание чуть более подробно вникнуть в суть вопроса, имеет смысл глянуть вот эту [лекцию из малого ШАДа,](https://habrahabr.ru/company/yandex/blog/307260/) прочитанную школьникам. 